In [1]:
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification
import gradio as gr
import torch
import string
from nltk.corpus import stopwords

path = './trained_over_model'
# model = BertForSequenceClassification.from_pretrained(path)
model = AutoModelForSequenceClassification.from_pretrained(path)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model

In [4]:
from nltk.stem import PorterStemmer
import re

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def text_processing(text):
    processed_text = text.lower()
    processed_text = re.sub(r'<[^>]+>', '', processed_text)
    processed_text = re.sub(r'[^a-zA-Z0-9]', ' ', processed_text)
    words = list(set(processed_text.split()))
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    processed_text = ' '.join(words)
    processed_text = re.sub(r'\d+', 'NUM', processed_text)
    return processed_text

def predict_sentiment(text):
    model.to(device)
    text = text_processing(text)
    inputs = tokenizer(text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=-1)

    return 'Negative Review 🥵 🟥' if prediction.item() == 0 else 'Positive Review 🥰 🟩'

# iface = gr.Interface(fn=predict_sentiment, inputs="text", outputs="text")
# iface.launch()

In [5]:
import gradio as gr

a = ["First of all the snug is a very nice place. The only problem is their food was not incredible and I feel like the prices could be a little cheaper. I would not go back because me and my daughter got sick afterwards, but if you are in to trying  new things you should definitely go because my friend went and said she didn't get sick at all, so I guess it just depends. Just make sure to be very cautious.",
 'Some of the best drinks ever, with the quickest and most efficient service. I love a bar that is nice, quiet enough where you can have a conversation, and has small bites. This bar also was so incredibly efficient and had a lot of space, they used their outdoor space efficiently and it was warm enough in the middle of winter. On a Friday night the staff was able to seat our party of 5 in 2 minutes which was pleasant. Then we used a QR code to order and got each round of our drinks in under 10 minutes. Will definitely come back!!!!',
 "Stoped into this place yesterday, Tuesday, March 12 at 4 pm hoping to enjoy some happy hour drinks and have early dinner. Unfortunately, the experience left much to be desired. Firstly, the nitro margarita tasted like artificial citrus and was far from enjoyable.Despite hearing great things about the food, we were disappointed when a young female did not allow us to order chips and salsa, despite nachos being on the menu. The inconsistency was baffling, especially when the app they use to order clearly showed nachos with chips and salsa. Even when I was willing to pay $15 for nachos with just the chips and salsa, the staff refused to accommodate. $15 dollars for chips, what business denies this request?? They lack traditional service, as everything is ordered through a QR code at the table. While I usually don't mind this approach, especially when the staff's attitude is subpar. They were unfriendly and unaccommodating throughout our visit.The lack of flexibility, unwillingness to accommodate requests and the unsatisfactory cocktails was our sign to take our business elsewhere. While the space itself is nice and locals may enjoy the food because it's convenient for them, I would never recommend this place or return myself out of choice. The 20% fair wage surcharge seems disproportionate when considering the level of service provided. As someone who typically tips generously, I believe in fair compensation for staff, but it should be reflective of the quality of service received.",
 "This was a nice place to meet friends. There was a nice selection of drinks (including non-alcoholic ones!). My friend liked her Cherry Blossom made with non-alcoholic whiskey. It got loud and crowded and there isn't a big food selection. We tried the burger (good) and the fried chicken sandwich (not good), both with tater tots. The tots were crispy but not hot and needed salt. The chicken sandwich was a fail. First it was marinated, which I never like. It's as if it has an unpleasant fug. Don't do that. Just leave it fresh! Then the slaw should be tangy and crisp. Nope. This was dank and flavorless. And it said something about habanero so I was expecting spice. It was spice level ZERO. So that was also disappointing. The entire sandwich was flavorless except for the fug of whatever they marinated the chicken in before they fried it. Horrible. The whole thing was a no. Even the pickles? Could not taste them.Service was fine but the guy came by 75 times before our friend got there so we sort of felt we were being no rushed.",
 'Food: I got the impossible burger and fries. They were good for bar food, but I would not come back here for dinner as it was not as inventive as I typically want my menus to be. The service was friendly! Ambiance was nice, good for a drink with an intimate group since it is dimly lit and can get loud. This place is known for their cocktails but they were decent for me, and nothing really stuck out. A chill bar to go to but nothing memorable about it in my opinion!',
 "Great place for drinks!! In addition to a meal of course too. Tried the black walnut drink which was smooth and delicious. They're open early ..so great hangout for happy hour if you have the Monday blues.",
 "Have been wanting to come here for the longest time but either the wait it too long or they're closed. Finally made it and was worth the wait. Tried prickly pear for my drink. Soooooo good! Want to go back for more cocktails for sure!! Went with the tomato burrata salad with pesto. Very good as well. Would totally commmmeee back!!",
 "My husband and I were walking around the Fillmore District and came upon The Snug while on a search for brunch and honestly, a super yummy smash burger from the City.  The Fillmore District has so many spots to tuck into, but don't let this one pass you completely by.  We went on a day that it just so happened to be raining, so warm drinks and comfort food were in order!  This spot hits the mark when it comes to cozy & cocktails, with a beautiful bar that takes front stage & is most def. the vibe.  Highlights for us were the Crispy Brussel Sprouts (with roasted garlic miso aioli, $12)  and the Wagyu Double Smash Cheeseburger (double wagyu beef patties, American cheese, dijonaise, arugula, challah bun, $18) with tater tots! Hit the spot for our cravings, and I would say worthy of consideration for a hidden gem to add to the list of smash burger favorites.  Some smash burgers leave you wanting more, some leave your hands and shirt in a bit of a mess, and some you wish just stuck to the basics.  This one will have you leaving satisfied and wanting a second round!To note: they don't accept cash tips, rather include a 20% Fair Wage Surcharge to the check - very cool and worth the quick, google research of how this helps their workers! All in all, for a chill afternoon with great service, unique cocktails and delicious bites, enjoy the Snug.  (we were able to find parking on a side street, so if driving, check out a quick search of spots to park near Fillmore venue).  Enjoy!",
 "Came here for a friends birthday on the weekend night and was seating at one of the larger tables. It seems the upstairs section was not open for the night, and it was packed. However, even though it was packed, we received subpar service. My friend who is a regular there was a bit surprised at the lack of timely service. I didn't think the staff was overwhelmed, with some of them seeing to take a lax approach to the night when I personally would have thought they would be checking in on tables or checking for patrons that were looking for a server to place order and waiting minutes.I got one of their coffee espressos (the name might be off), but its was delicious and worth the expectations that my friend had set.I enjoyed my time here and probably would come back!",
 "I definitely kept it snug at The Snug! It's up to par with other cocktail bars in the area but I do feel like they have their own hits and misses.My companion took me here after I got off work. The spot is spacious and clean with seating at the bar and tables in one area. Or a communal table and window seating on the other side. They also have plenty of outdoor seating. We chose to sit by the windows which beckoned natural light and a place to people watch. My main gripe is the 20% automated tip considering there is zero service from start to finish. You sit down, order and pay using their QR code, and receive your food and drinks from a runner--no interactions whatsoever which is sucky.Their QR code leads you to their lunch, dinner, and libations menu. Its comfort American and California fare is on the pricier end. Plus uniquely crafted cocktails. I enjoyed my drink and the Tots my companion and I shared--super crispy, not soggy, and good quality. We got Ketchup and Yuzu Mayo on the side. The Yuzu Mayo was phenomenally creamy and you could clearly taste the citrusy flavor. One more is they charge for their sauces so it does get costly!This is a solid option for a drink and bites for those in the Pac Heights or the surrounding area. Parking is on the street off Fillmore or surrounding areas. Ideal for pairs or small and large groups as it's inviting and roomy. Personally, I won't become a regular as the food and drinks are overpriced considering there's no service and a mandatory surcharge, but I'll come back on special occasions!",
 '3.5 rounded to 4 stars. Nice location, pretty great ambiance. You could order your drinks by scanning the QR code, we had a hard time finding the code because it was pretty small and taped to the side of the table. We had the Chamomile gin and Coconut daiquiri, both were pretty meh in my opinion. I was expecting the drinks to have more depth of flavor but they just straight up tasted bitter. I think the Chamomile gin was a bit lighter and had more flavor. In addition, each drink came up to $18 including tax and the gratitude fee which was on a more pricey side.']

with gr.Blocks() as demo1:
  with gr.Row():
    with gr.Column():
      review = gr.Textbox(label='Review')
      pred_btn = gr.Button(value='Predict')

      # get_btn = gr.Button(value='Get')

    with gr.Column():
      Bsentiment = gr.Textbox(label="BERT Sentiment")
      BLsentiment = gr.Textbox(label="BiLSTM Sentiment")
      RoBERTasentiment = gr.Textbox(label="BERT Sentiment")

    pred_btn.click(predict_sentiment, inputs=review, outputs=Bsentiment)

  review_list = gr.Examples(examples=a,
                            inputs=[review]
                            ,label="Review list"
                            )

demo1.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
